In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

class PenguinModel:
    def __init__(self):
        self.model = None
        self.label_encoder_species = LabelEncoder()
        self.label_encoder_sex = LabelEncoder()
        self.is_fitted = False

    def read_data(self, filepath):
        """
        Чтение данных из CSV файла и предобработка
        """
        data = pd.read_csv(filepath)
        # Оставляем только необходимые столбцы
        data = data[['Species', 'Culmen Length (mm)', 'Culmen Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)', 'Sex', 'Island']]
        data.dropna(inplace=True)

        # Преобразование категориальных переменных
        data['Species'] = self.label_encoder_species.fit_transform(data['Species'])
        data['Sex'] = self.label_encoder_sex.fit_transform(data['Sex'])
        data = pd.get_dummies(data, columns=['Island'], drop_first=True)

        X = data.drop(columns='Species')
        y = data['Species']

        return X, y

    def fit(self, X, y):
        """
        Обучение модели логистической регрессии
        """
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model = LogisticRegression(max_iter=1000)
        self.model.fit(X_train, y_train)
        self.is_fitted = True

        # Оценка точности на тестовой выборке
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")
        return accuracy

    def predict(self, X):
        """
        Предсказание классов для новых данных
        """
        if not self.is_fitted:
            raise ValueError("The model must be fitted before predicting.")
        return self.model.predict(X)


In [ ]:
# Функция демонстрации модели с визуализацией
def demonstrate_penguin_model(filepath):
    # Шаг 1. Инициализация модели и чтение данных
    model = PenguinModel()
    X, y = model.read_data(filepath)

    # Шаг 2. Обучение модели
    accuracy = model.fit(X, y)
    print(f"Точность модели: {accuracy:.2f}")

    # Шаг 3. Прогнозирование
    y_pred = model.predict(X)

    # Шаг 4. Визуализация результатов
    # Преобразуем признаки в 2D пространство с помощью PCA для визуализации
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    # Построение графика
    plt.figure(figsize=(10, 6))

    # Отобразим фактические классы
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', marker='o', edgecolor='k', alpha=0.7, label='Фактические данные')

    # Отобразим предсказанные классы
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_pred, cmap='cool', marker='x', edgecolor='k', alpha=0.5, label='Предсказанные данные')

    plt.title('Фактические vs Предсказанные классы пингвинов')
    plt.xlabel('Первая главная компонента')
    plt.ylabel('Вторая главная компонента')
    plt.colorbar(label='Классы пингвинов')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()


# Пример использования
filepath = 'penguins_size.csv'  # Убедитесь, что у вас есть этот файл в той же директории
demonstrate_penguin_model(filepath)